# Implementation of Basic Interactor Demo

Found here:

https://github.com/bokeh/bokeh/blob/0.12.7/examples/howto/notebook_comms/Jupyter%20Interactors.ipynb

In [14]:
from ipywidgets import interact
import numpy as np
import pandas as pd
import json

In [27]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.palettes import Spectral5
output_notebook()

Loading BokehJS ...

In [15]:
def create_dataframes(json_metadata_path):
    """Construct dataframes with the needed metadata attached."""
    # Read the metadata json file constructed by ISASetup.py
    with open(json_metadata_path, 'r') as f:
        metadata = json.load(f)
    # create a dataframes for each csv found
    dataFrame_list = []
    for study in metadata['studies']:
        # Store the doi/link
        for publication in study['publications']:
            doi = publication['doi']
        for assay in study['assays']:
            for filename in assay['dataFiles']:
                data = pd.read_csv(filename['name'])
                data['doi'] = doi
                dataFrame_list.append(data)
    return pd.concat(dataFrame_list, ignore_index=True)

data = create_dataframes('data/nmr_metadata.json')

In [21]:
data.head()

,Al_concentration,Al_ppm,CI_concentration,OH_concentration,counter_ion,doi,temperature,wavelength
0,0.56,NaN,2.59,4.04,Na+,http://collections.mun.ca/cdm/compoundobject/c...,25,619.0
1,0.84,NaN,3.83,6.12,Na+,http://collections.mun.ca/cdm/compoundobject/c...,25,619.5
2,1.12,NaN,4.75,7.88,Na+,http://collections.mun.ca/cdm/compoundobject/c...,25,619.5
3,1.62,NaN,7.23,11.86,Na+,http://collections.mun.ca/cdm/compoundobject/c...,25,619.5
4,2.08,NaN,9.24,15.25,Na+,http://collections.mun.ca/cdm/compoundobject/c...,25,620.0


In [28]:
columns = sorted(data.columns)
SIZES = list(range(6, 22, 3))
COLORS = Spectral5

In [31]:
# Get the values that are discrete.
discrete = [x for x in columns if data[x].dtype == object]

# Get the continuous
continuous = [x for x in columns if x not in discrete]

# Maybe some of the numerical values are the same?
quantileable = [x for x in continuous if len(data[x].unique()) > 20]

In [33]:
def create_figure():
    
    # Declare the x and y axis values to be those input from the selectors.
    xv = data[x_selector.value].values
    yv = data[y_selector.value].values
    
    # Declare variables for the titles, get the title from the selectors.
    x_title = x_selector.value.title()
    y_title = y_selector.value.title()
    
    # Declare a dictionary to hold the data.
    kw = dict()
    
    # Use some low-level accesss to set the axis ranges.
    # The 'x_range' is an accepted dictionary key for the plotting object to be used.
    if x_selector.value in discrete:
        kw['x_range'] = sorted(set(xv))
    if y_selector.value in discrete:
        kw['y_range'] = sorted(set(yv))
    
    # Set the title.
    kw['title'] = "{1} vs {2}".format(x_title, y_title)
    
    # Create the figure object
    fig = figure(
        plot_height=600, 
        plot_width=800,
        tools='pan,box_zoom,reset', 
        **kw,
    )
    
    # Set the axis labels.
    fig.xaxis.axis_label = x_title
    fig.yaxis.axis_label = y_title
    
    # Set the default point size.
    sz = 9
    
    if size.value != 'None':
        # Use pandas qcut, which can bin data sets, make sure to ignore duplicates
        # so we get the desired spread.
        groups = pd.qcut(df[size.value].values, len(SIZES), duplicates='drop')
        sz = [SIZES[xx] for xx in groups.codes]
        
    # Set the default color
    c = "#31AADE"
    
    if color.value != 'None':
        groups = pd.qcut(df[color.value].values, len(COLORS), duplicates='drop')
        
    # Draw the actual points on the plot.
    fig.circle(
        x=xv,
        y=yv,
        color=c,
        size=sz,
        line_color='white',
        alpha=0.6,
        hover_color='white',
        hover_alpha=0.5
    )
    
    # Then return this figure.
    return fig

In [38]:
columns

['Al_concentration',
 'Al_ppm',
 'CI_concentration',
 'OH_concentration',
 'counter_ion',
 'doi',
 'temperature',
 'wavelength']

In [39]:
interact(
    update,
    x_selector=columns,
    y_selector=columns,
)
# x = Select(title='X-Axis', value='mpg', options=columns)
# x.on_change('value', update)

ValueError: cannot find widget or abbreviation for argument: 'attr'

In [34]:
def update(attr, old, new):
    layout.childeren[1] = create_figure()

In [36]:
fig = create_figure()

NameError: name 'x_selector' is not defined

In [35]:
show(fig, notebook_handle=True)

NameError: name 'fig' is not defined

In [9]:
interact(update, f=["sin", "cos", "tan"], w=(0,100), A=(1,5), phi=(0, 20, 0.1))

<function __main__.update>